In [1]:
import json
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Circle
import tf_silent
from pinn import PINN
from network import Network
from optimizer_sep_losses import L_BFGS_B

In [3]:
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.interpolate import interp1d

def enhanced_wake_metrics(x_grid, y_grid, u, v, p, model_name=""):
    """
    Comprehensive wake region analysis for Kármán vortices detection
    """
    # Define wake region (adjust as needed)
    wake_x_start = 0.6  # just behind cylinder
    wake_x_end = 1.5
    wake_y_center = 0.5
    wake_y_half_width = 0.3
    
    # Create wake mask
    wake_mask = ((x_grid >= wake_x_start) & (x_grid <= wake_x_end) & 
                 (y_grid >= wake_y_center - wake_y_half_width) & 
                 (y_grid <= wake_y_center + wake_y_half_width))
    
    metrics = {}
    
    # 1. Minimum velocity in wake (your current metric)
    u_wake = u[wake_mask]
    metrics['min_u_wake'] = float(np.min(u_wake))
    metrics['mean_u_wake'] = float(np.mean(u_wake))
    
    # 2. Vorticity magnitude in wake region
    # Calculate vorticity: ω = ∂v/∂x - ∂u/∂y
    dy = y_grid[1, 0] - y_grid[0, 0]
    dx = x_grid[0, 1] - x_grid[0, 0]
    
    du_dy = np.gradient(u, dy, axis=0)
    dv_dx = np.gradient(v, dx, axis=1)
    vorticity = dv_dx - du_dy
    
    vorticity_wake = vorticity[wake_mask]
    metrics['max_vorticity_magnitude'] = float(np.max(np.abs(vorticity_wake)))
    metrics['mean_vorticity_magnitude'] = float(np.mean(np.abs(vorticity_wake)))
    
    # 3. Velocity fluctuation analysis (indicator of unsteady behavior)
    # Check for alternating positive/negative regions
    centerline_idx = np.argmin(np.abs(y_grid[:, 0] - 0.5))
    u_centerline = u[centerline_idx, :]
    x_centerline = x_grid[centerline_idx, :]
    
    # Focus on wake region along centerline
    wake_centerline_mask = (x_centerline >= wake_x_start) & (x_centerline <= wake_x_end)
    u_wake_centerline = u_centerline[wake_centerline_mask]
    x_wake_centerline = x_centerline[wake_centerline_mask]
    
    # Count zero crossings (sign changes) in wake
    sign_changes = np.sum(np.diff(np.sign(u_wake_centerline)) != 0)
    metrics['wake_sign_changes'] = int(sign_changes)
    
    # 4. Recirculation length estimation
    # Find where u becomes positive again after the cylinder
    recirculation_length = 0
    for i, x_val in enumerate(x_wake_centerline):
        if u_wake_centerline[i] > 0.01:  # threshold for positive flow
            recirculation_length = x_val - 0.6  # distance from cylinder rear
            break
    metrics['recirculation_length'] = float(recirculation_length)
    
    # 5. Wake asymmetry metric
    # Compare upper and lower wake regions
    upper_wake_mask = ((x_grid >= wake_x_start) & (x_grid <= wake_x_end) & 
                       (y_grid >= wake_y_center) & (y_grid <= wake_y_center + wake_y_half_width))
    lower_wake_mask = ((x_grid >= wake_x_start) & (x_grid <= wake_x_end) & 
                       (y_grid >= wake_y_center - wake_y_half_width) & (y_grid <= wake_y_center))
    
    u_upper = np.mean(u[upper_wake_mask])
    u_lower = np.mean(u[lower_wake_mask])
    metrics['wake_asymmetry'] = float(abs(u_upper - u_lower))
    
    return metrics

# Apply to all your models
def analyze_all_models():
    """Analyze all saved models with enhanced metrics"""
    
    # Load the training summary
    with open('model_data/track_sep_losses/all_training_summary_2.json', 'r') as f:
        results = json.load(f)
    
    enhanced_results = []
    
    for result in results:
        model_path = result['model_filename']
        loss_weights = result['loss_weights']
        cyl_weight = loss_weights[-1]  # Last index is cylinder boundary weight
        
        print(f"Analyzing model with cylinder weight: {cyl_weight}")
        
        # Load model and predict
        # from tensorflow.keras.models import load_model # tf.keras.models.load_model
        network = tf.keras.models.load_model(model_path)
        
        # Create inference grid
        x = np.linspace(0, 2, 300)
        y = np.linspace(0, 1, 300)
        x_grid, y_grid = np.meshgrid(x, y)
        xy_flat = np.stack([x_grid.flatten(), y_grid.flatten()], axis=-1)
        
        # Predict
        u_v_p = network.predict(xy_flat, batch_size=len(xy_flat))
        u = u_v_p[..., 0].reshape(x_grid.shape)
        v = u_v_p[..., 1].reshape(x_grid.shape)
        p = u_v_p[..., 2].reshape(x_grid.shape)
        
        # Calculate enhanced metrics
        enhanced_metrics = enhanced_wake_metrics(x_grid, y_grid, u, v, p, 
                                               model_name=f"cyl_weight_{cyl_weight}")
        
        # Combine with existing results
        enhanced_result = result.copy()
        enhanced_result.update(enhanced_metrics)
        enhanced_results.append(enhanced_result)
    
    return enhanced_results

# Run the analysis
enhanced_results = analyze_all_models()

Analyzing model with cylinder weight: 5.0


OSError: SavedModel file does not exist at: model_data/track_sep_losses/eqn5_cyl5/implicit_noroi_LC_eqn5cyl5.keras/{saved_model.pbtxt|saved_model.pb}

In [2]:
def analyze_component_losses():
    """Analyze how cylinder boundary loss changes with weight"""
    
    with open('model_data/track_sep_losses/all_training_summary_2.json', 'r') as f:
        results = json.load(f)
    
    loss_analysis = []
    
    for result in results:
        loss_weights = result['loss_weights']
        cyl_weight = loss_weights[-1]  # Cylinder boundary weight
        model_name = result['model_filename'].split('/')[-1].replace('.keras', '')
        
        # Load component losses
        component_loss_file = f"model_data/track_sep_losses/eqn{int(loss_weights[0])}_cyl{int(cyl_weight)}/{model_name}_component_losses.npy"
        
        try:
            component_losses = np.load(component_loss_file, allow_pickle=True)
            
            # Extract cylinder boundary loss (last component, index -1)
            cyl_losses = []
            for iteration_losses in component_losses:
                if iteration_losses is not None and len(iteration_losses) > 0:
                    cyl_losses.append(iteration_losses[-1])  # Last component is cylinder
            
            if cyl_losses:
                initial_cyl_loss = cyl_losses[0]
                final_cyl_loss = cyl_losses[-1]
                min_cyl_loss = min(cyl_losses)
                
                reduction_ratio = (initial_cyl_loss - final_cyl_loss) / initial_cyl_loss
                
                loss_analysis.append({
                    'cyl_weight': cyl_weight,
                    'initial_cyl_loss': initial_cyl_loss,
                    'final_cyl_loss': final_cyl_loss,
                    'min_cyl_loss': min_cyl_loss,
                    'loss_reduction_ratio': reduction_ratio,
                    'cyl_loss_history': cyl_losses
                })
        
        except FileNotFoundError:
            print(f"Component loss file not found for {model_name}")
    
    return loss_analysis

# Run component loss analysis
loss_analysis = analyze_component_losses()

In [3]:
def create_comprehensive_analysis_plots(enhanced_results, loss_analysis):
    """Create comprehensive plots for model comparison"""
    
    # Extract data for plotting
    cyl_weights = [r['loss_weights'][-1] for r in enhanced_results]
    
    # Create subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Wake metrics vs cylinder weight
    min_u_wake = [r['min_u_wake'] for r in enhanced_results]
    max_vorticity = [r['max_vorticity_magnitude'] for r in enhanced_results]
    recirculation_length = [r['recirculation_length'] for r in enhanced_results]
    
    axes[0, 0].plot(cyl_weights, min_u_wake, 'bo-', linewidth=2, markersize=8)
    axes[0, 0].set_xlabel('Cylinder Boundary Weight')
    axes[0, 0].set_ylabel('Min U Velocity in Wake')
    axes[0, 0].set_title('Wake Velocity vs Boundary Weight')
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(cyl_weights, max_vorticity, 'ro-', linewidth=2, markersize=8)
    axes[0, 1].set_xlabel('Cylinder Boundary Weight')
    axes[0, 1].set_ylabel('Max Vorticity Magnitude')
    axes[0, 1].set_title('Vorticity vs Boundary Weight')
    axes[0, 1].grid(True)
    
    axes[0, 2].plot(cyl_weights, recirculation_length, 'go-', linewidth=2, markersize=8)
    axes[0, 2].set_xlabel('Cylinder Boundary Weight')
    axes[0, 2].set_ylabel('Recirculation Length')
    axes[0, 2].set_title('Recirculation Length vs Boundary Weight')
    axes[0, 2].grid(True)
    
    # 2. Component loss analysis
    if loss_analysis:
        loss_cyl_weights = [la['cyl_weight'] for la in loss_analysis]
        final_cyl_losses = [la['final_cyl_loss'] for la in loss_analysis]
        reduction_ratios = [la['loss_reduction_ratio'] for la in loss_analysis]
        
        axes[1, 0].plot(loss_cyl_weights, final_cyl_losses, 'mo-', linewidth=2, markersize=8)
        axes[1, 0].set_xlabel('Cylinder Boundary Weight')
        axes[1, 0].set_ylabel('Final Cylinder Loss')
        axes[1, 0].set_title('Cylinder Boundary Loss vs Weight')
        axes[1, 0].set_yscale('log')
        axes[1, 0].grid(True)
        
        axes[1, 1].plot(loss_cyl_weights, reduction_ratios, 'co-', linewidth=2, markersize=8)
        axes[1, 1].set_xlabel('Cylinder Boundary Weight')
        axes[1, 1].set_ylabel('Loss Reduction Ratio')
        axes[1, 1].set_title('Cylinder Loss Reduction vs Weight')
        axes[1, 1].grid(True)
        
        # Loss convergence curves
        for la in loss_analysis:
            iterations = range(len(la['cyl_loss_history']))
            axes[1, 2].plot(iterations, la['cyl_loss_history'], 
                           label=f"Weight {la['cyl_weight']}", linewidth=2)
        
        axes[1, 2].set_xlabel('Iteration')
        axes[1, 2].set_ylabel('Cylinder Boundary Loss')
        axes[1, 2].set_title('Cylinder Loss Convergence')
        axes[1, 2].set_yscale('log')
        axes[1, 2].legend()
        axes[1, 2].grid(True)
    
    plt.tight_layout()
    plt.savefig('comprehensive_model_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

# Create the comprehensive analysis
create_comprehensive_analysis_plots(enhanced_results, loss_analysis)

NameError: name 'enhanced_results' is not defined

In [ ]:
def karman_vortex_score(x_grid, y_grid, u, v):
    """
    Calculate a score indicating likelihood of Kármán vortex formation
    Higher score = more likely to have vortex shedding
    """
    # Calculate vorticity
    dy = y_grid[1, 0] - y_grid[0, 0]
    dx = x_grid[0, 1] - x_grid[0, 0]
    
    du_dy = np.gradient(u, dy, axis=0)
    dv_dx = np.gradient(v, dx, axis=1)
    vorticity = dv_dx - du_dy
    
    # Define wake region
    wake_mask = ((x_grid >= 0.6) & (x_grid <= 1.5) & 
                 (y_grid >= 0.2) & (y_grid <= 0.8))
    
    vorticity_wake = vorticity[wake_mask]
    
    # Score components:
    # 1. Vorticity strength
    vorticity_score = np.std(vorticity_wake) * 10
    
    # 2. Alternating pattern (check for positive and negative vorticity regions)
    positive_vorticity = np.sum(vorticity_wake > 0.1)
    negative_vorticity = np.sum(vorticity_wake < -0.1)
    alternating_score = min(positive_vorticity, negative_vorticity) / 100
    
    # 3. Recirculation strength
    recirculation_score = abs(np.min(u[wake_mask])) * 10
    
    # Combined score
    karman_score = vorticity_score + alternating_score + recirculation_score
    
    return {
        'karman_score': float(karman_score),
        'vorticity_component': float(vorticity_score),
        'alternating_component': float(alternating_score),
        'recirculation_component': float(recirculation_score)
    }

# Add Kármán score to your analysis
def add_karman_scores(enhanced_results):
    """Add Kármán vortex scores to existing results"""
    for result in enhanced_results:
        # Re-load and analyze the model (you might want to cache this)
        # ... (similar to previous loading code)
        karman_metrics = karman_vortex_score(x_grid, y_grid, u, v)
        result.update(karman_metrics)
    
    return enhanced_results

In [ ]:
def generate_summary_report(enhanced_results, loss_analysis):
    """Generate a comprehensive summary report"""
    
    print("="*60)
    print("COMPREHENSIVE MODEL ANALYSIS REPORT")
    print("="*60)
    
    # Sort by cylinder weight
    enhanced_results_sorted = sorted(enhanced_results, key=lambda x: x['loss_weights'][-1])
    
    print(f"{'Cyl Weight':<12} {'Min U Wake':<12} {'Max Vorticity':<15} {'Recirc Length':<15} {'Kármán Score':<12}")
    print("-"*70)
    
    for result in enhanced_results_sorted:
        cyl_weight = result['loss_weights'][-1]
        min_u = result['min_u_wake']
        max_vort = result.get('max_vorticity_magnitude', 0)
        recirc_len = result.get('recirculation_length', 0)
        karman_score = result.get('karman_score', 0)
        
        print(f"{cyl_weight:<12.1f} {min_u:<12.4f} {max_vort:<15.4f} {recirc_len:<15.4f} {karman_score:<12.4f}")
    
    # Find best model for vortex formation
    if 'karman_score' in enhanced_results[0]:
        best_model = max(enhanced_results, key=lambda x: x['karman_score'])
        print(f"\nBest model for Kármán vortices: Cylinder weight = {best_model['loss_weights'][-1]}")
        print(f"Kármán score: {best_model['karman_score']:.4f}")
    
    # Component loss analysis summary
    if loss_analysis:
        print(f"\n{'Cyl Weight':<12} {'Initial Loss':<15} {'Final Loss':<15} {'Reduction %':<12}")
        print("-"*60)
        for la in sorted(loss_analysis, key=lambda x: x['cyl_weight']):
            reduction_pct = la['loss_reduction_ratio'] * 100
            print(f"{la['cyl_weight']:<12.1f} {la['initial_cyl_loss']:<15.6f} {la['final_cyl_loss']:<15.6f} {reduction_pct:<12.2f}")

# Generate the report
generate_summary_report(enhanced_results, loss_analysis)